# Библиотеки Python для DS (семинары)

Урок 8. На практике использование методов генерации признаков

В вашем распоряжении набор данных о качестве сна Sleep_health_and_lifestyle_dataset.csv.
Ваша задача — подготовить данные к моделированию, сгенерировать новые признаки, применить методы отбора признаков для улучшения качества предсказательной модели и подобрать оптимальные гиперпараметры для модели регрессии.

1. Предобработка данных:
— Очистите данные от пропусков и аномалий.
— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

2. Генерация новых признаков:
— Создайте комбинированные признаки, которые могут помочь улучшить модель.
— Используйте полиномиальные признаки для числовых переменных.

3. Отбор признаков:
— Примените несколько методов отбора признаков (например, RFE, SelectKBest).
— Сравните качество модели до и после отбора признаков.

4. Подбор гиперпараметров:
— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.
— Оцените, как изменение гиперпараметров влияет на качество предсказаний.

## Выполнение домашней работы

##### Используем предоставленый к заданию файл fetch_california_housing.xlsx

In [25]:
# Библиотеки для работы с датасетом и графиками
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# Загружаем датасет
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')

##### 1. Предобработка данных:

— Очистите данные от пропусков и аномалий.

— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

In [19]:
# Выводим дата фрейм
df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,No Disorder
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,No Disorder
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,No Disorder
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


Этот фрейм данных содержит следующие столбцы:

1. Person ID (Идентификатор для каждого человека)
2. Gender (Пол человека (мужчина/женщина))
3. Age (Возраст человека в годах)
4. Occupation (Род занятий или профессия человека)
5. Sleep Duration (hours) (Количество часов, которые человек спит в сутки)
6. Quality of Sleep (scale: 1-10) (Субъективная оценка качества сна в диапазоне от 1 до 10)
7. Physical Activity Level (minutes/day) (Количество минут, которые человек ежедневно уделяет физической активности)
8. Stress Level (scale: 1-10) (Субъективная оценка уровня стресса, испытываемого человеком, в диапазоне от 1 до 10)
9. BMI Category (Категория ИМТ человека (например, недостаточный вес, нормальный, избыточный вес с избыточной массой тела))
10. Blood Pressure (systolic/diastolic) (Измерение артериального давления человека, показывающее превышение систолического давления над диастолическим давлением)
11. Heart Rate (bpm) (Частота сердечных сокращений человека в состоянии покоя в ударах в минуту)
12. Daily Steps (Количество шагов, которые человек делает за день)
13. Sleep Disorder (Наличие или отсутствие у человека нарушений сна (отсутствие таковых, бессонница, апноэ во сне))

In [4]:
# Выводим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


Из общей информации следует:
1. Всего 13 колонок, DataFrame содержит 374 записи, от 0 до 373
2. Нулевые значения отсуствуют
3. Есть пропущеные записи в колонке Sleep Disorder
4. Колонки: Person ID, Age, Quality of Sleep, Physical Activity Level, Stress Level, Heart Rate, Daily Steps находятся в целочисленых значениях int64
5. Колонки: Sleep Duration находится в дробном формате float64
6. Колонки: Gender, Occupation, BMI Category, Blood Pressure, Sleep Disorder являются объектами object

In [5]:
# Статистический анализ числовых переменных
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Person ID,374.0,187.500000,108.108742,1.0,94.25,187.5,280.75,374.0
Age,374.0,42.184492,8.673133,27.0,35.25,43.0,50.00,59.0
Sleep Duration,374.0,7.132086,0.795657,5.8,6.40,7.2,7.80,8.5
Quality of Sleep,374.0,7.312834,1.196956,4.0,6.00,7.0,8.00,9.0
Physical Activity Level,374.0,59.171123,20.830804,30.0,45.00,60.0,75.00,90.0
Stress Level,374.0,5.385027,1.774526,3.0,4.00,5.0,7.00,8.0
Heart Rate,374.0,70.165775,4.135676,65.0,68.00,70.0,72.00,86.0
Daily Steps,374.0,6816.844920,1617.915679,3000.0,5600.00,7000.0,8000.00,10000.0


На основе представленных статистических данных неприрывных признаков можно сделать следующие выводы:

1. Age (Возраст человека в годах): 
- Средний возраст человека на уровне 43.00
- Миниманая 27.00
- Максимальная 59.00
- Медиана равна 42.18
- Отклонение состовляет 8.67
2. Sleep Duration (hours) (Количество часов, которые человек спит в сутки):
- Средние количество часов которое человек спит в сутки на уровне 7.20
- Миниманая 5.80
- Максимальная 8.50
- Медиана равна 7.13
- Отклонение состовляет 0.79
3. Quality of Sleep (scale: 1-10) (Субъективная оценка качества сна в диапазоне от 1 до 10):
- Средняя оценка качества сна на уровне 7.00
- Миниманая 4.00
- Максимальная 9.00
- Медиана равна 7.31
- Отклонение состовляет 1.19
4. Physical Activity Level (minutes/day) (Количество минут, которые человек ежедневно уделяет физической активности):
- Средние количество минут которые человек ежедневно уделяет физической активности на уровне 60.00
- Миниманая 30.00
- Максимальная 90.00
- Медиана равна 59.17
- Отклонение состовляет 20.83
5. Stress Level (scale: 1-10) (Субъективная оценка уровня стресса, испытываемого человеком, в диапазоне от 1 до 10):
- Средние субъективная оценка уровня стреса на уровне 5.00
- Миниманая 3.00
- Максимальная 8.00
- Медиана равна 5.38
- Отклонение состовляет 5.38
6. Heart Rate (bpm) (Частота сердечных сокращений человека в состоянии покоя в ударах в минуту):
- Средняя частота сердечных сокращений на уровне 70.00
- Миниманая 65.00
- Максимальная 86.00
- Медиана равна 70.16
- Отклонение состовляет 4.13
7. Daily Steps (Количество шагов, которые человек делает за день):
- Средние количество шагов на уровне 7000.00
- Миниманая 3000.00
- Максимальная 10000.00
- Медиана равна 6816.84
- Отклонение состовляет 1617.91

In [ ]:
# Статистический анализ объектных признаков
df.describe(include='object').T

,count,unique,top,freq
Gender,374,2,Male,189
Occupation,374,11,Nurse,73
BMI Category,374,4,Normal,195
Blood Pressure,374,25,130/85,99
Sleep Disorder,155,2,Sleep Apnea,78


1. Gender (Пол человека (мужчина/женщина)):
- В данных присутствуют записи о поле у 374 индивидов.
- Имеются всего две уникальные категории пола.
- Наиболее часто встречающийся пол - Мужчина (Male), с частотой 189 раз.

2. Occupation (Род занятий или профессия человека)
- Для 374 индивидов имеются данные о профессии.
- В данных присутствует 11 уникальных профессий.
- Профессия 'Nurse' (Медсестра/Медбрат) встречается чаще всего - 73 раза.

3. BMI Category (Категория ИМТ человека (например, недостаточный вес, нормальный, избыточный вес с избыточной массой тела))
- Информация о категории индекса массы тела (ИМТ) присутствует для всех 374 индивидов.
- Обнаружено 4 уникальные категории ИМТ.
- Наиболее распространенная категория ИМТ - 'Normal' (Норма), встречается 195 раз.

4. Blood Pressure (systolic/diastolic) (Измерение артериального давления человека, показывающее превышение систолического давления над диастолическим давлением)
- Записи об артериальном давлении имеются для каждого из 374 индивидов.
- Выявлено 25 различных значений артериального давления.
- Самый распространенный уровень артериального давления среди участников - 130/85, зафиксирован 99 раз.

5. Sleep Disorder (Наличие или отсутствие у человека нарушений сна (отсутствие таковых, бессонница, апноэ во сне))
- Информация о соновых расстройствах есть для всех 374 индивидов.
- В данных есть 3 уникальных состояния соновых расстройств.
- Наиболее частая запись - 'No Disorder' (Нет расстройства), она встречается в 219 случаях


In [7]:
# Проверяем на дубликаты
print("Количество дубликатов:")
print(df.duplicated().sum())

Количество дубликатов:
0


Дубликаты отсутствуют.

In [8]:
# Смотрим ещё раз тип данных
df.dtypes

Person ID                    int64
Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
Blood Pressure              object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder              object
dtype: object

In [9]:
# Проверка на нулевые значения
df.isnull().sum()

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64

In [10]:
# Проверка на NaN значения
df.isnull().any().any()

True

По итогу проверок на NaN значения, пропущеные значения находим 219 пропущеных NaN значений.

In [12]:
# Заменяем пропущенные (NaN) значения.
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('No Disorder')

In [13]:
# Проверка на NaN значения
df.isnull().any().any()

False

Получаем отсуствие пропущеных NaN значений

In [23]:
# One-Hot Encoding
df_encoder = pd.get_dummies(df, columns=['Gender', 'Occupation', 'Blood Pressure', 'Sleep Disorder'], drop_first=True)

df_encoder

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Gender_Male,...,Blood Pressure_131/86,Blood Pressure_132/87,Blood Pressure_135/88,Blood Pressure_135/90,Blood Pressure_139/91,Blood Pressure_140/90,Blood Pressure_140/95,Blood Pressure_142/92,Sleep Disorder_No Disorder,Sleep Disorder_Sleep Apnea
0,1,27,6.1,6,42,6,Overweight,77,4200,True,...,False,False,False,False,False,False,False,False,True,False
1,2,28,6.2,6,60,8,Normal,75,10000,True,...,False,False,False,False,False,False,False,False,True,False
2,3,28,6.2,6,60,8,Normal,75,10000,True,...,False,False,False,False,False,False,False,False,True,False
3,4,28,5.9,4,30,8,Obese,85,3000,True,...,False,False,False,False,False,True,False,False,False,True
4,5,28,5.9,4,30,8,Obese,85,3000,True,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,59,8.1,9,75,3,Overweight,68,7000,False,...,False,False,False,False,False,False,True,False,False,True
370,371,59,8.0,9,75,3,Overweight,68,7000,False,...,False,False,False,False,False,False,True,False,False,True
371,372,59,8.1,9,75,3,Overweight,68,7000,False,...,False,False,False,False,False,False,True,False,False,True
372,373,59,8.1,9,75,3,Overweight,68,7000,False,...,False,False,False,False,False,False,True,False,False,True


In [26]:
# Создание экземпляра LabelEncoder
label_encoder = LabelEncoder()

# Применение Label Encoding к каждому категориальному столбцу
df_encoder['BMI Category'] = label_encoder.fit_transform(df_encoder['BMI Category'])

df_encoder

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Gender_Male,...,Blood Pressure_131/86,Blood Pressure_132/87,Blood Pressure_135/88,Blood Pressure_135/90,Blood Pressure_139/91,Blood Pressure_140/90,Blood Pressure_140/95,Blood Pressure_142/92,Sleep Disorder_No Disorder,Sleep Disorder_Sleep Apnea
0,1,27,6.1,6,42,6,3,77,4200,True,...,False,False,False,False,False,False,False,False,True,False
1,2,28,6.2,6,60,8,0,75,10000,True,...,False,False,False,False,False,False,False,False,True,False
2,3,28,6.2,6,60,8,0,75,10000,True,...,False,False,False,False,False,False,False,False,True,False
3,4,28,5.9,4,30,8,2,85,3000,True,...,False,False,False,False,False,True,False,False,False,True
4,5,28,5.9,4,30,8,2,85,3000,True,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,59,8.1,9,75,3,3,68,7000,False,...,False,False,False,False,False,False,True,False,False,True
370,371,59,8.0,9,75,3,3,68,7000,False,...,False,False,False,False,False,False,True,False,False,True
371,372,59,8.1,9,75,3,3,68,7000,False,...,False,False,False,False,False,False,True,False,False,True
372,373,59,8.1,9,75,3,3,68,7000,False,...,False,False,False,False,False,False,True,False,False,True


In [ ]:
# Установка стиля графиков
sns.set(style="whitegrid")

# Визуализация распределения нескольких выбранных числовых переменных
selected_columns = ['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'target']

plt.figure(figsize=(15, 10))

for i, column in enumerate(selected_columns):
    plt.subplot(2, 3, i + 1)
    sns.histplot(df[column], kde=True, bins=30)
    plt.title(f'Распределение {column}')
    plt.xlabel(column)
    plt.ylabel('Частота')

plt.tight_layout()
plt.show()

In [ ]:
# Визуализация корреляции между выбранными переменными
correlation_matrix = df[selected_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Корреляционная матрица')
plt.show()

In [ ]:
correlation_matrix = df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, mask=mask, cmap='coolwarm', vmax=1, vmin=-1, center=0, 
            square=True, linewidths=.5, annot_kws={"size": 10}, 
            cbar_kws={"shrink": 0.8, "orientation": "vertical", "ticks": [-1, -0.5, 0, 0.5, 1]})
plt.title('Матрица корреляций между числовыми переменными')
plt.show()


##### 2. Генерация новых признаков:

— Создайте комбинированные признаки, которые могут помочь улучшить модель.

— Используйте полиномиальные признаки для числовых переменных.

In [13]:
df_home = df
# Извлечём diagnosis из признаков
X = df_home.drop('target', axis=1)
# Определим целевую переменную
target = df_home['target']

In [14]:
# Стандартизация данных
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)

# Создание нового DataFrame со стандартизированными данными
df_scaled = pd.DataFrame(X_scaler, columns=X.columns)

In [15]:
# Определяем переменные для обучения
df_irus_scaled = df_scaled
y = target

# Разделяем данные на обучающий и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(df_irus_scaled, y, test_size=0.2, random_state=42)

# Выводим размеры
print("Размер обучающего набора:", X_train.shape)
print("Размер тестового набора:", X_test.shape)

Размер обучающего набора: (16512, 8)
Размер тестового набора: (4128, 8)


##### 3. Отбор признаков:

— Примените несколько методов отбора признаков (например, RFE, SelectKBest).

— Сравните качество модели до и после отбора признаков.

##### 4. Подбор гиперпараметров:

— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.

— Оцените, как изменение гиперпараметров влияет на качество предсказаний.